In [1]:
#import packages
import pandas as pd
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
#modify import path
import sys
sys.path.append('../our_analysis')

#import our code
from hybrid_mle import fit_hybrid_mixed_model, fit_hybrid_mixed_dynamic_model


/Users/AdelynnMeow/opt/anaconda3/envs/stan_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#Set WD
notebook_dir = os.getcwd()

In [4]:
#load dataset
# df = pd.read_csv("../dataset/all_subjects.csv") #our dataset
df = pd.read_csv("../dataset/beh_noslow.csv") #i'm fitting to replicate the paper.


## some notes about using all_subjects:
# If you get errors you may need to rename the fields of allsubjects becuase the original paper dataset (beh_noslow.csv) and our dataset (all_subjects) have slightly different capitalizations of headers.
# If you get additional errors filter out the NaNs from the df before sending them to the processing function. 

#MLE takes time the first simple run was quite fast (~10min), but expect longer depending on your CPU and the complexity of the model you're running

story_trials = df[df["condition"] == "story"]
abstract_trials=df[df["condition"] == "abstract"]

# Fit Hybrid Model
We're currently fitting using the provided stan model. I made modifications to ensure we could just pass dfs instead of raw csvs. 

We are using a mixed effects approach to estimate the weight allowing it to vary across participants. Not necessary as we could fix all effects (the pset is unclear what they're really asking for here), but this provides more nuance to the analysis. 

Note we likely won't be very close to the reported table in the paper as estimated individually and then pooled across all participants.

We can rerun this to get results closer to the paper by setting the priors in the stan model. Look at the upper/lower parameter bounds in the stan file (sdn_hw2/our_analysis/hybrid_mixed.stan). We want a decent width for the parameter search but we can make them a bit closer to the actual paper. @TODO by anyone: it would be good to check the supplemental to see if they added a note about what the priors were

In [5]:
story_results_df, params = fit_hybrid_mixed_model(
    data_df=story_trials,
    stan_file="../our_analysis/hybrid_mixed.stan",
    output_file="story_params.csv"
)

abstract_results_df, params = fit_hybrid_mixed_model(
    data_df=abstract_trials,
    stan_file="../our_analysis/hybrid_mixed.stan",
    output_file="abstract_params.csv"
)

ValueError: no such file /Users/AdelynnMeow/Documents/GitHub/sdn_hw2/our_analysis/hybrid_mixed.stan

In [24]:
#store the results in a clean table we can use for the assignment
results =[] 

for df in [story_results_df,abstract_results_df]:
    params=df.drop(columns='w').iloc[0].to_dict()
    params['w_mean'] = df['w'].mean()
    params['w_std'] = df['w'].std()
    results.append(params)

results_df = pd.DataFrame(results)
results_df.head()
## we should probably just save this as a csv here. It has the participant field which we can drop in later formatting.
results_df.to_csv("hybrid_model_params_abstract_fixp0.csv", index=False)

# Let's do some ablation:

Here we're going to start fitting with fixed parameters. To do this I added a modified fit function that allows us to prefix certain parameters. This should work to finish the next few parts

In [6]:
# PARAM_NAMES = ('alpha1', 'alpha2', 'lmbd', 'beta1', 'beta2', 'p', 'w')


## USE THE PARAM NAMES ABOVE. ONLY INPUT THE PARAMS you are FIXING and the value you are fixing it to. 
fixed_param = {
    'lmbd':0.99
}

story_results_df, params, logli = fit_hybrid_mixed_dynamic_model(
    data_df=story_trials, #note the trial type
    stan_file="../our_analysis/dynamic_hybrid_mixed.stan",
    output_file="story_params_p0.csv",   ##change the output csv name so you don't overwrite your work
    fixed_params = fixed_param,
    return_logli = True
)


22:25:33 - cmdstanpy - INFO - compiling stan file /Users/AdelynnMeow/Documents/GitHub/sdn_hw2/our_analysis/dynamic_hybrid_mixed.stan to exe file /Users/AdelynnMeow/Documents/GitHub/sdn_hw2/our_analysis/dynamic_hybrid_mixed


KeyboardInterrupt: 

In [10]:
# PARAM_NAMES = ('alpha1', 'alpha2', 'lmbd', 'beta1', 'beta2', 'p', 'w')


## USE THE PARAM NAMES ABOVE. ONLY INPUT THE PARAMS you are FIXING and the value you are fixing it to. 
fixed_param = {
    'p':0
}

story_results_df, params, logli = fit_hybrid_mixed_dynamic_model(
    data_df=story_trials, #note the trial type
    stan_file="../our_analysis/dynamic_hybrid_mixed.stan",
    output_file="story_params_p0.csv",   ##change the output csv name so you don't overwrite your work
    fixed_params = fixed_param,
    return_logli = True
)

abstract_results_df, params, logli = fit_hybrid_mixed_dynamic_model(
    data_df=abstract_trials, #note the trial type
    stan_file="../our_analysis/dynamic_hybrid_mixed.stan",
    output_file="abstract_params_p0.csv",   ##change the output csv name so you don't overwrite your work
    fixed_params = fixed_param,
    return_logli = True
)


01:46:05 - cmdstanpy - INFO - Chain [1] start processing
01:46:08 - cmdstanpy - INFO - Chain [1] done processing
01:46:08 - cmdstanpy - INFO - Chain [1] start processing
01:46:11 - cmdstanpy - INFO - Chain [1] done processing
01:46:11 - cmdstanpy - INFO - Chain [1] start processing
01:46:14 - cmdstanpy - INFO - Chain [1] done processing
01:46:14 - cmdstanpy - INFO - Chain [1] start processing
01:46:18 - cmdstanpy - INFO - Chain [1] done processing
01:46:18 - cmdstanpy - INFO - Chain [1] start processing
01:46:22 - cmdstanpy - INFO - Chain [1] done processing
01:46:22 - cmdstanpy - INFO - Chain [1] start processing
01:46:25 - cmdstanpy - INFO - Chain [1] done processing
01:46:25 - cmdstanpy - INFO - Chain [1] start processing
01:46:27 - cmdstanpy - INFO - Chain [1] done processing
01:46:27 - cmdstanpy - INFO - Chain [1] start processing
01:46:30 - cmdstanpy - INFO - Chain [1] done processing
01:46:30 - cmdstanpy - INFO - Chain [1] start processing
01:46:34 - cmdstanpy - INFO - Chain [1]

Optimized parameters:
OrderedDict([('lp__', -7548.79), ('alpha1_free[1]', 0.0118232), ('alpha2_free[1]', 0.841627), ('lmbd_free[1]', 0.791962), ('beta1_free[1]', 8.71315), ('beta2_free[1]', 2.03594), ('w_free[1]', 0.999971), ('w_free[2]', 0.357969), ('w_free[3]', 0.999825), ('w_free[4]', 0.194871), ('w_free[5]', 0.512003), ('w_free[6]', 0.999985), ('w_free[7]', 0.319743), ('w_free[8]', 0.76313), ('w_free[9]', 0.484527), ('w_free[10]', 0.945393), ('w_free[11]', 0.658748), ('w_free[12]', 0.218452), ('w_free[13]', 0.265848), ('w_free[14]', 0.493386), ('w_free[15]', 0.104513), ('w_free[16]', 0.751408), ('w_free[17]', 0.489282), ('w_free[18]', 0.336135), ('w_free[19]', 0.0027174), ('w_free[20]', 0.0297261), ('w_free[21]', 0.884865), ('w_free[22]', 0.462494), ('w_free[23]', 0.387197), ('w_free[24]', 1.19835e-06), ('w_free[25]', 0.405711), ('w_free[26]', 0.586498), ('w_free[27]', 0.847658), ('w_free[28]', 9.91143e-06), ('w_free[29]', 0.211656), ('w_free[30]', 0.999974), ('w_free[31]', 5.54262

02:38:08 - cmdstanpy - INFO - Chain [1] start processing
02:38:09 - cmdstanpy - INFO - Chain [1] done processing
02:38:09 - cmdstanpy - INFO - Chain [1] start processing
02:38:12 - cmdstanpy - INFO - Chain [1] done processing
02:38:12 - cmdstanpy - INFO - Chain [1] start processing
02:38:13 - cmdstanpy - INFO - Chain [1] done processing
02:38:13 - cmdstanpy - INFO - Chain [1] start processing
02:38:16 - cmdstanpy - INFO - Chain [1] done processing
02:38:16 - cmdstanpy - INFO - Chain [1] start processing
02:38:18 - cmdstanpy - INFO - Chain [1] done processing
02:38:18 - cmdstanpy - INFO - Chain [1] start processing
02:38:22 - cmdstanpy - INFO - Chain [1] done processing
02:38:22 - cmdstanpy - INFO - Chain [1] start processing
02:38:24 - cmdstanpy - INFO - Chain [1] done processing
02:38:24 - cmdstanpy - INFO - Chain [1] start processing
02:38:26 - cmdstanpy - INFO - Chain [1] done processing
02:38:26 - cmdstanpy - INFO - Chain [1] start processing
02:38:28 - cmdstanpy - INFO - Chain [1]

Optimized parameters:
OrderedDict([('lp__', -7617.97), ('alpha1_free[1]', 0.16562), ('alpha2_free[1]', 0.457856), ('lmbd_free[1]', 0.449236), ('beta1_free[1]', 8.32195), ('beta2_free[1]', 3.12149), ('w_free[1]', 0.999989), ('w_free[2]', 0.00023359), ('w_free[3]', 0.999964), ('w_free[4]', 0.374231), ('w_free[5]', 0.404122), ('w_free[6]', 0.500251), ('w_free[7]', 0.362429), ('w_free[8]', 0.999906), ('w_free[9]', 0.301299), ('w_free[10]', 0.202918), ('w_free[11]', 0.709151), ('w_free[12]', 0.664474), ('w_free[13]', 6.7245e-05), ('w_free[14]', 0.134586), ('w_free[15]', 0.49586), ('w_free[16]', 0.135665), ('w_free[17]', 0.347182), ('w_free[18]', 0.523094), ('w_free[19]', 0.588471), ('w_free[20]', 0.99996), ('w_free[21]', 0.804923), ('w_free[22]', 0.231083), ('w_free[23]', 0.608072), ('w_free[24]', 4.29322e-05), ('w_free[25]', 0.385647), ('w_free[26]', 0.829662), ('w_free[27]', 0.24754), ('w_free[28]', 0.802782), ('w_free[29]', 0.639435), ('w_free[30]', 0.720161), ('w_free[31]', 0.523759), (

In [15]:
story_results_df.head()

## You should save these as a csv. Snag the logli from above as well!

,participant,condition,alpha1,alpha2,lmbd,beta1,beta2,p,w
0,1,story,0.011823,0.841627,0.791962,8.71315,2.03594,0,0.999971
1,2,story,0.011823,0.841627,0.791962,8.71315,2.03594,0,0.357969
2,3,story,0.011823,0.841627,0.791962,8.71315,2.03594,0,0.999825
3,7,story,0.011823,0.841627,0.791962,8.71315,2.03594,0,0.194871
4,10,story,0.011823,0.841627,0.791962,8.71315,2.03594,0,0.512003


In [16]:
abstract_results_df.head()

,participant,condition,alpha1,alpha2,lmbd,beta1,beta2,p,w
0,5,abstract,0.16562,0.457856,0.449236,8.32195,3.12149,0,0.999989
1,6,abstract,0.16562,0.457856,0.449236,8.32195,3.12149,0,0.000234
2,8,abstract,0.16562,0.457856,0.449236,8.32195,3.12149,0,0.999964
3,9,abstract,0.16562,0.457856,0.449236,8.32195,3.12149,0,0.374231
4,11,abstract,0.16562,0.457856,0.449236,8.32195,3.12149,0,0.404122


In [13]:
#store the results in a clean table we can use for the assignment
results =[] 

for df in [story_results_df,abstract_results_df]:
    params=df.drop(columns='w').iloc[0].to_dict()
    params['w_mean'] = df['w'].mean()
    params['w_std'] = df['w'].std()
    results.append(params)

results_df = pd.DataFrame(results)
results_df.head()
## we should probably just save this as a csv here. It has the participant field which we can drop in later formatting.
results_df.to_csv("hybrid_model_params_fix_p0.csv", index=False)

In [17]:
params

{'participant': 5,
 'condition': 'abstract',
 'alpha1': 0.16562,
 'alpha2': 0.457856,
 'lmbd': 0.449236,
 'beta1': 8.32195,
 'beta2': 3.12149,
 'p': 0,
 'w_mean': 0.5316883034833333,
 'w_std': 0.3212041309271306}